Vamos a hacer análisis exploratorio de un dataset de tweets.

Es muy importante siempre investigar acerca del dataset antes de explorarlo (e.g. leer repositorios o papers asociados, etc.). ¿Quién lo recopiló? ¿Con qué propósito? ¿Cómo se recopiló? Etc...

-----------------------

Tarea: responder donde dice **PREGUNTA**

## Configuración del entorno

In [ ]:
!pip install -qU datasets spacy nltk scikit-learn watermark

In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp datasets,spacy,nltk,sklearn,numpy,pandas,tqdm,matplotlib

## Carga de datos

Vamos a cargar el dataset con la librería de Hugging Face `datasets` pero lo vamos a convertir a un DataFrame de pandas. Más adelante vamos a trabajar con `datasets` directamente.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emotion")

¿Qué pinta tiene?

In [ ]:
dataset

In [ ]:
dataset["train"].features

In [ ]:
dataset["train"][0]

In [ ]:
# Convertimos los labels a strings por comodidad
int2label = dataset["train"].features["label"].int2str
dataset = dataset.map(lambda example: {"label_str": int2label(example["label"])}, remove_columns=["label"])
dataset = dataset.rename_column("label_str", "label")

In [ ]:
dataset["train"][0]

In [ ]:
# Convertimos a pandas por comodidad
import pandas as pd
pd.options.display.max_colwidth = 300

dfs = {split: dataset[split].to_pandas() for split in dataset.keys()}
#del dataset

In [ ]:
dfs["train"].sample(3, random_state=33)

## Análisis exploratorio

Vamos a ir respondiendo preguntas que uno típicamente se hace al explorar un dataset como este, que luego se va a usar para alguna tarea de NLP.

### 1. ¿Cómo es la distribución de clases?

In [ ]:
for split, df in dfs.items():
    print(split)
    print(df["label"].value_counts(normalize=False))
    print()

### 2. ¿Cómo es la longitud de los textos? ¿Hay documentos anormalmente largos o cortos?

Para responder esto necesitamos una manera de contar palabras / tokens / unidades de texto. Es decir, necesitamos **tokenizar**.

In [ ]:
# A veces alcanza con contar espacios:
num_words = {}
for split, df in dfs.items():
    num_words[split] = df["text"].str.count(" ") + 1

for split, df in dfs.items():
    print(split)
    print(num_words[split].describe())
    print()

In [ ]:
# O contar caracteres:
num_chars = {}
for split, df in dfs.items():
    num_chars[split] = df["text"].str.len()

for split, df in dfs.items():
    print(split)
    print(num_chars[split].describe())
    print()

Otras veces nos gustaría ser más cuidados a la hora de separar en palabras, por ejemplo, en inglés podríamos separar "don't" en "do" y "n't", o "I'm" en "I" y "'m".

Para esto podemos usar tokenizadores informados por el lenguaje, como los de la librería `spacy`.

In [ ]:
# Usando spacy:
from spacy.lang.en import English

nlp = English()
tokenizer = nlp.tokenizer

example = dfs["train"]["text"].iloc[0]
tokens_example = tokenizer(example)

print(example)
print([token.text for token in tokens_example])

In [ ]:
# Usando tokenizer.pipe podemos correrlo para una lista de documentos:
num_tokens = {}

for split, df in dfs.items():
    generator_ = tokenizer.pipe(df["text"], batch_size=50)
    num_tokens[split] = pd.Series([len(doc) for doc in generator_])

for split, df in dfs.items():
    print(split)
    print(num_tokens[split].describe())
    print()

Si estamos trabajando con tweets, donde los emojis y los hashtags son importantes, quizás sea mejor usar un tokenizador especializado en tweets:

In [ ]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
# strip_handles=False: elimina los @user
# reduce_len=True: reduce los caracteres repetidos a 3 e.g. "aaaaaa" -> "aaa"

example = dfs["train"]["text"].iloc[0]
tokens_example = tokenizer.tokenize(example)

print(example)
print(tokens_example)


**PREGUNTA 1**: ¿Por qué motivo querríamos usar `reduce_len=True`?

In [ ]:
# Veamos cuáles son los ejemplos más largos y más cortos:
import textwrap

tokenizer = TweetTokenizer(strip_handles=False, reduce_len=False)

for split, df in dfs.items():
    df["num_tokens"] = df["text"].apply(lambda x: len(tokenizer.tokenize(x)))

for split, df in dfs.items():
    short_texts = df.sort_values("num_tokens", ascending=True).head(3)["text"].values
    long_texts = df.sort_values("num_tokens", ascending=False).head(3)["text"].values
    print("##", split)
    print("# Short:")
    for text in short_texts:
        print("- \t", textwrap.fill(text, 120))
    print("# Long:")
    for text in long_texts:
        print("- \t", textwrap.fill(text, 120))
    print()


In [ ]:
dfs["train"].sort_values("num_tokens", ascending=False).head(3)

### 3. ¿Hay caracteres raros o inesperados?

In [ ]:
# Por ejemplo, caracteres html como &amp; se pueden convertir a su forma original:

mask = dfs["train"]["text"].str.contains("&amp;")
print(dfs["train"][mask].shape)

In [ ]:
import html

example = dfs["train"][mask]["text"].iloc[0]

print(example)
print(html.unescape(example))

In [ ]:
# O caracteres whitespace no identificados como tales:
mask = dfs["train"]["text"].str.contains("\\\\n") # tienen "\n" literales
print(dfs["train"][mask].shape)

example = dfs["train"][mask]["text"].iloc[1]
print(example)
print(example.replace("\\n", "\n"))

### 4. ¿Hay documentos duplicados?


In [ ]:
# buscar duplicados:
for split, df in dfs.items():
    print(split)
    print(df["text"].duplicated().sum())
    print()

In [ ]:
mask = dfs["train"]["text"].duplicated(keep=False)
dfs["train"][mask].sort_values("text")

**PREGUNTA 2**: ¿por qué motivo podrían estar duplicados estos tweets?

### 4. ¿Hay documentos raros?

"raro" es un término subjetivo, pero podríamos pensar en documentos que son muy cortos, muy largos, con contenido inesperado, etc. O si hay una variable respuesta, documentos con errores de anotación.

Una manera sencilla y bastante general de detectar documentos raros en tareas de clasificación es corriendo un modelo sencillo y viendo los documentos que más pérdida generan, i.e. los que el modelo no puede clasificar bien.

Vamos a usar una regresión logística para esto. Más adelante vamos a analizar esto con much más detalle!

In [ ]:
print(dfs["train"].shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(max_features=150)
clf = LogisticRegression(max_iter=1000)
class2int = {label: i for i, label in enumerate(dfs["train"]["label"].unique())}
int2class = {i: label for label, i in class2int.items()}

X_train = vectorizer.fit_transform(dfs["train"]["text"])
y_train = dfs["train"]["label"].map(class2int)

clf.fit(X_train, y_train)

**PREGUNTA 3** ¿qué representa cada fila y cada columna de `X_train`?

In [ ]:
# Ejemplos con mayor pérdida (medida como -log(probabilidad_clase_correcta)):
import numpy as np

y_pred_proba = clf.predict_proba(X_train)
loss = -np.log(y_pred_proba[range(len(y_train)), y_train])
y_pred = clf.predict(X_train)

dfs["train"]["pred"] = [int2class[i] for i in y_pred]
dfs["train"]["loss"] = loss

dfs["train"].sort_values("loss", ascending=False).head(8)

### 5. ¿Cómo es la distribución de palabras?

Para esto es fundamental (1) preprocesar el texto y (2) tokenizarlo. La manera en la que hagamos esto depende directamente del análisis que queramos hacer.

Cosas a definir: ¿queremos diferenciar mayúsculas y minúsculas? ¿Queremos eliminar _stopwords_? ¿Queremos eliminar puntuación? ¿Queremos lematizar o hacer stemming?

In [ ]:
# Empecemos usando el tokenizador de tweets de nltk.
# Para cada documento queremos un vector del tamaño del vocabulario con la
# cantidad de veces que aparece cada palabra.
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(strip_handles=False, reduce_len=True)
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize)

X_train = vectorizer.fit_transform(dfs["train"]["text"])
df_vocab_train = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())

print(df_vocab_train.shape)
df_vocab_train.head()

**PREGUNTA 4**: ¿cuántas palabras hay en el vocabulario?

In [ ]:
# palabras más y menos frecuentes:
vocab_freq = df_vocab_train.sum().sort_values(ascending=False)

print(vocab_freq.head(10))
print(vocab_freq.tail(10))

In [ ]:
# Eliminando stopwords (palabras frecuentes con poca carga semántica),
# mayúsculas, mentions y punctuación (pero sin eliminar hashtags!):
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
import string

stop_words = stopwords.words("english")
print(stop_words[:5])

print(string.punctuation)

def preprocess_text(text: str) -> str:
    """Limpia antes de tokenizar."""
    text = text.lower()
    text = text.replace("\\\\n", " ")
    text = html.unescape(text)
    exclude = {'#', "'", "@"}
    text = ''.join(char for char in text if char not in string.punctuation or char in exclude)
    return text

example = "I'm a #tweet with @user and a link: https://t.co/1234"
print(example)
print(preprocess_text(example))

In [ ]:
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
vectorizer = CountVectorizer(
    tokenizer=tokenizer.tokenize, stop_words=stop_words, preprocessor=preprocess_text
)

X_train = vectorizer.fit_transform(dfs["train"]["text"])
df_vocab_train = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())

print(df_vocab_train.shape)
df_vocab_train.head()

In [ ]:
vocab_freq = df_vocab_train.sum().sort_values(ascending=False)
print(vocab_freq.head(20))

**PREGUNTA 5**: ¿Cómo tratarían las tildes en español para este análisis exploratorio?

In [ ]:
# Palabras más populares de cada clase:
df_data = df_vocab_train.copy()
df_data["CLASE"] = dfs["train"]["label"]
# pusimos CLASE para que no se confunda con una palabra del vocabulario
df_data = df_data.groupby("CLASE").sum().T
df_data.sort_values("joy", ascending=False).head(10)

In [ ]:
# visualizamos las palabras más populares de cada clase:
import matplotlib.pyplot as plt

labels = list(df_data.columns)
n = 20

for label in labels:
    df_plot = df_data[label].sort_values(ascending=False).head(n)
    df_plot.plot(kind="bar", title=label, figsize=(6, 2))
    plt.show()

In [ ]:
# Ahora, % de documentos en los que aparece cada palabra, por clase:
df_data = df_vocab_train.astype(bool).astype(int).copy()
df_data["CLASE"] = dfs["train"]["label"]
df_data = df_data.groupby("CLASE").mean().T

for label in labels:
    df_plot = df_data[label].sort_values(ascending=False).head(n)
    df_plot.plot(kind="bar", title=label, figsize=(6, 2))
    plt.show()

In [ ]:
# Esto se puede seguir mejorando...
# Más adelante vamos a ver cómo encontrar las palabras más _discriminativas_ entre clases

**PREGUNTA 6**: ¿Cuándo puede ser distinto analizar la frecuencia y el % de documentos en los que aparece una palabra?

**PREGUNTA 7**: ¿Cómo mejorar visualmente estos gráficos?

### 6. Stemming, lematización, y regex: ejemplos de uso

- **Stemming**: Reducción de palabras a su raíz base.
- **Lematización**: Transformación de palabras a su forma canónica.
- **Regex**: expresiones regulares para identificar patrones en el texto.

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer

nlp = English()
tokenizer = nlp.tokenizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

examples = [
    "Speaking words of wisdom, let it be.",
    "His palms are sweaty, knees weak, arms are heavy.",
    "The mice were running through the house, searching for food.",
]

for example in examples:
    tokens = [token.text for token in tokenizer(example)]
    stemmed = [stemmer.stem(word) for word in tokens]
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]

    print("Stemming", stemmed)
    print("Lemmatized", lemmatized)
    print()

# TODO regex_words = re.findall(r'\b\w+mente\b', texto)

**PREGUNTA 8** ¿En qué casos serviría usar stemming / lematización para un problema de clasificación? Ante la duda, ¿cómo puedo definir si sirve o no?

Algunas expresiones regulares comunes:

* "." : Matchea cualquier caracter excepto '\n'
* "^" y "$": Matchean el comienzo y el final de un string
* "[]": Matchea el set de caracteres que se encuentren dentro de los corchetes (r"l[ao]s" machea "las" y "los")
* \d: Matchea digitos; equivalente a [0-9].
* \D: Matchea caracteres que NO sean digitos; equivalente a [^0-9].
* \s: Matchea espacios en blanco; equivalente a [ \t\n\r\f\v].
* \S: Matchea espacios que NO esten en blanco; equivalente a [^ \t\n\r\f\v].
* \w: Matchea caracteres alfanuméricos; equivalente a [a-zA-Z0-9_].
* \W: Matchea caracteres que NO sean alfanuméricos; equivalente a[^a-zA-Z0-9_].
* a|b: Matchea "a" o "b"

Para repeticiones de patrones:
* "+": Matchea 1 o mas ocurrencias
* "*": Matchea 0 o mas ocurrencias
* "?": Matchea 0 o 1 ocurrencia
* "{n, m}": Matchea entre n y m ocurrencias
* "\\": Permite matchear caracteres especiales

Para más info ver: https://docs.python.org/3.1/library/re.html#re-syntax

In [ ]:
# Algunos ejemplos:
import re

texto = "las cámaras y los libros sobre laos de luis alberto"
patron = r"l[ao]s"
resultados = re.findall(patron, texto)
print(texto)
print(patron)
print(resultados)
print()

texto = "Mi número es 12345 y tu número es 67890."
patron = r"\d+"
resultados = re.findall(patron, texto)
print(texto)
print(patron)
print(resultados)
print()

texto = "123 un pasito palante maría, 123 un pasito patrás"
patron = r"\D+"
resultados = re.findall(patron, texto)
print(texto)
print(patron)
print(resultados)
print()

# Buscamos precios compuestos por 2 o 3 dígitos, opcionalmente seguidos de un espacio y la palabra "USD" o "usd"
texto = "El precio es 100USD, o tal vez 50 USD o 250usd."
patron = r"(\d{2,3})\s?[USD|usd]"
resultados = re.findall(patron, texto)
print(texto)
print(patron)
print(resultados)
print()